<a href="https://colab.research.google.com/github/MiguelMendozaG/CANproject/blob/6_features/LSTM_Autoencoder_Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from sklearn.model_selection import train_test_split
from keras import optimizers
DATA_SPLIT_PCT = 0.2
SEED = 123
max_size = 100
n_features = 6
lookback = max_size

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')
model_dir = '/content/gdrive/My Drive/'

Mounted at /content/gdrive/


In [3]:
car_model = "Colab Notebooks/Car"

In [4]:
model_dir + car_model

'/content/gdrive/My Drive/Colab Notebooks/Car'

In [5]:
data = np.load(model_dir + car_model + '/recorrido.npy')

In [6]:
def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(0,lookback):
            # Gather past records upto the lookback period
            t.append(X[[(i+j)], :])
        output_X.append(t)
        output_y.append(y[i+lookback])
    return output_X, output_y


In [7]:
def flatten(X):
    '''
    Flatten a 3D array.
    
    Input
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    
    Output
    flattened_X  A 2D array, sample x features.
    '''
    flattened_X = np.empty((X.shape[0], X.shape[2]) )  # sample x features array.
    for i in range(X.shape[0]):
      flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)

def scale(X, scaler):
    '''
    Scale 3D array.

    Inputs
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    scaler       A scaler object, e.g., sklearn.preprocessing.StandardScaler, sklearn.preprocessing.normalize
    
    Output
    X            Scaled 3D array.
    '''
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])
        
    return X

In [8]:
X_train, X_test = train_test_split(np.array(data), test_size=DATA_SPLIT_PCT, random_state=SEED)
X_train, X_validation = train_test_split(X_train, test_size=DATA_SPLIT_PCT, random_state=SEED)
X_train_original_shape = X_train.shape
X_test_original_shape = X_test.shape
X_validation_original_shape = X_validation.shape

In [9]:
X_train_reshape = X_train.reshape(X_train.shape[0], lookback, n_features)
X_train_float = np.empty((X_train_reshape.shape[0], X_train_reshape.shape[1], X_train_reshape.shape[2]))
X_train_float = np.array(X_train_reshape, dtype= np.float64)

X_train_float_test = np.empty((X_train_reshape.shape[0], X_train_reshape.shape[1], X_train_reshape.shape[2]))
X_train_float_test = np.array(X_train_reshape, dtype= np.float64)

X_test_reshape = X_test.reshape(X_test.shape[0], lookback, n_features)
X_test_float = np.empty((X_test_reshape.shape[0], X_test_reshape.shape[1], X_test_reshape.shape[2]))
X_test_float = np.array(X_test_reshape, dtype= np.float64)

X_validation_reshape = X_validation.reshape(X_validation.shape[0], lookback, n_features)
X_validation_float = np.empty((X_validation_reshape.shape[0], X_validation_reshape.shape[1], X_validation_reshape.shape[2]))
X_validation_float = np.array(X_validation_reshape, dtype= np.float64)

print (X_train_reshape.shape)
print (X_test_reshape.shape)
print (X_validation_reshape.shape)

(24788, 100, 6)
(7747, 100, 6)
(6198, 100, 6)


In [10]:
X_train_split = flatten(X_train_reshape)
X_train_array = np.array(np.hsplit( X_train_split, 6))

In [11]:
# define min max scalers for each message
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
scaler3 = MinMaxScaler()
scaler4 = MinMaxScaler()
scaler5 = MinMaxScaler()
scaler6 = MinMaxScaler()

X_train_float[:,:,0] = scaler1.fit_transform(X_train_reshape[:,:,0])
X_train_float[:,:,1] = scaler2.fit_transform(X_train_reshape[:,:,1])
X_train_float[:,:,2] = scaler3.fit_transform(X_train_reshape[:,:,2])
X_train_float[:,:,3] = scaler4.fit_transform(X_train_reshape[:,:,3])
X_train_float[:,:,4] = scaler5.fit_transform(X_train_reshape[:,:,4])
X_train_float[:,:,5] = scaler6.fit_transform(X_train_reshape[:,:,5])

In [12]:
#Train statistics
print ("Msg 1")
print(X_train_float[:,:,0])
print (X_train_float[:,:,0].min(), X_train_float[:,:,0].max())

Msg 1
[[0.59090861 0.59090844 0.59090844 ... 0.59090852 0.68420848 0.684208  ]
 [0.59091065 0.59091065 0.59091065 ... 0.59090783 0.68420788 0.6842075 ]
 [0.59090835 0.59090835 0.59090852 ... 0.59090818 0.68420838 0.6842079 ]
 ...
 [0.59090826 0.59090826 0.59090826 ... 0.59090844 0.68420858 0.6842078 ]
 [0.59090826 0.59090844 0.59090844 ... 0.59090826 0.68420838 0.6842079 ]
 [0.59091186 0.59091177 0.59091177 ... 0.59091125 0.68421184 0.68421136]]
0.0 1.0


In [13]:
#transform test and validation with minmax statistics
X_test_float[:,:,0] = scaler1.transform(X_test_reshape[:,:,0])
X_test_float[:,:,1] = scaler2.transform(X_test_reshape[:,:,1])
X_test_float[:,:,2] = scaler3.transform(X_test_reshape[:,:,2])
X_test_float[:,:,3] = scaler4.transform(X_test_reshape[:,:,3])
X_test_float[:,:,4] = scaler5.transform(X_test_reshape[:,:,4])
X_test_float[:,:,5] = scaler6.transform(X_test_reshape[:,:,5])

X_validation_float[:,:,0] = scaler1.transform(X_validation_reshape[:,:,0])
X_validation_float[:,:,1] = scaler2.transform(X_validation_reshape[:,:,1])
X_validation_float[:,:,2] = scaler3.transform(X_validation_reshape[:,:,2])
X_validation_float[:,:,3] = scaler4.transform(X_validation_reshape[:,:,3])
X_validation_float[:,:,4] = scaler5.transform(X_validation_reshape[:,:,4])
X_validation_float[:,:,5] = scaler6.transform(X_validation_reshape[:,:,5])

In [14]:
#Test statistics
print ("Msg 1")
print(X_test_float[:,:,4])
print (X_test_float[:,:,4].min(), X_test_float[:,:,4].max())

Msg 1
[[0.74984747 0.74996949 0.75009152 ... 0.74996949 0.75009152 0.74972544]
 [0.84368517 0.8438072  0.84344112 ... 0.8438072  0.84344112 0.84356315]
 [0.84368517 0.8438072  0.84344112 ... 0.8438072  0.84344112 0.84356315]
 ...
 [0.8438072  0.84344112 0.84356315 ... 0.84344112 0.84356315 0.84368517]
 [0.8438072  0.84344112 0.84356315 ... 0.84344112 0.84356315 0.84368517]
 [0.84344112 0.84356315 0.84368517 ... 0.84356315 0.84368517 0.8438072 ]]
0.0 1.0


In [15]:
#Validation statistics
num = 5
print ("Msg 1")
print(X_validation_float[:,:,num])
print (X_validation_float[:,:,num].min(), X_validation_float[:,:,num].max())

Msg 1
[[1.00448750e-15 1.20626613e-15 1.40452024e-15 ... 1.20626613e-15
  1.40452024e-15 1.64859308e-15]
 [2.14285714e-01 2.14285714e-01 2.16269841e-01 ... 3.03571429e-01
  3.05555556e-01 3.07539683e-01]
 [0.00000000e+00 2.37023804e-16 4.84621162e-16 ... 2.37023804e-16
  4.84621162e-16 7.07546896e-16]
 ...
 [1.00448750e-15 1.20626613e-15 1.40452024e-15 ... 1.20626613e-15
  1.40452024e-15 1.64859308e-15]
 [7.07546896e-16 1.00448750e-15 1.20626613e-15 ... 1.00448750e-15
  1.20626613e-15 1.40452024e-15]
 [1.64859308e-15 1.99487693e-15 2.24599880e-15 ... 1.99487693e-15
  2.24599880e-15 2.43720388e-15]]
0.0 1.0000000000000004


In [16]:
timesteps = X_train_float.shape[1]
n_features = X_train_float.shape[2]
lr = 0.0001
print(timesteps)
print(n_features)

100
6


In [17]:
lstm_autoencoder = Sequential()
# Encoder
lstm_autoencoder.add(LSTM(32, activation='relu', input_shape=(timesteps, n_features), return_sequences=True))
lstm_autoencoder.add(LSTM(16, activation='relu', return_sequences=False))
lstm_autoencoder.add(RepeatVector(timesteps))
# Decoder
lstm_autoencoder.add(LSTM(16, activation='relu', return_sequences=True))
lstm_autoencoder.add(LSTM(32, activation='relu', return_sequences=True))
lstm_autoencoder.add(TimeDistributed(Dense(n_features)))

lstm_autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 32)           4992      
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                3136      
_________________________________________________________________
repeat_vector (RepeatVector) (None, 100, 16)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 16)           2112      
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 32)           6272      
_________________________________________________________________
time_distributed (TimeDistri (None, 100, 6)            198       
Total params: 16,710
Trainable params: 16,710
Non-trainable params: 0
____________________________________________________

In [18]:
adam = optimizers.Adam(lr)
lstm_autoencoder.compile(loss='mse', optimizer=adam)
# fit model
lstm_autoencoder_history = lstm_autoencoder.fit(X_train_float, X_train_float, validation_data=(X_validation_float, X_validation_float),
                                                verbose=1, epochs=300, batch_size=100).history
yhat = lstm_autoencoder.predict(X_test_float, verbose=0)

plt.plot(lstm_autoencoder_history['loss'], linewidth=2, label='Train')
plt.plot(lstm_autoencoder_history['val_loss'], linewidth=2, label='Valid')
plt.legend(loc='upper right')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

Epoch 1/300
248/248 [==============================] - 42s 169ms/step - loss: 0.1947 - val_loss: 0.1452
Epoch 2/300
248/248 [==============================] - 41s 165ms/step - loss: 0.1228 - val_loss: 0.1080
Epoch 3/300
248/248 [==============================] - 41s 167ms/step - loss: 0.0799 - val_loss: 0.0534
Epoch 4/300
248/248 [==============================] - 42s 169ms/step - loss: 0.0421 - val_loss: 0.0369
Epoch 5/300
248/248 [==============================] - 42s 170ms/step - loss: 0.0329 - val_loss: 0.0310
Epoch 6/300
248/248 [==============================] - 42s 169ms/step - loss: 0.0276 - val_loss: 0.0258
Epoch 7/300
248/248 [==============================] - 43s 173ms/step - loss: 0.0231 - val_loss: 0.0221
Epoch 8/300
248/248 [==============================] - 44s 177ms/step - loss: 0.0202 - val_loss: 0.0194
Epoch 9/300
248/248 [==============================] - 43s 174ms/step - loss: 0.0176 - val_loss: 0.0167
Epoch 10/300
248/248 [==============================] - 43s 174m

NameError: ignored